# Server - object detection 파이프라인

## 시도 1.
- 카메라가 열리면 object detection 시작
    - 카메라가 닫힐때까지 작업 유지
    - 카메라가 닫히면 종료
- 사람 트래킹
    - 사람 감지되면 녹화 시작
        - 사람감지 변수 : True
    - 사람 사라지면 녹화 비디오 저장
        - 사람감지 변수 : False

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
from time import time
from collections import defaultdict
from ultralytics.utils.plotting import Annotator, colors
import math 

from datetime import datetime  # 파일명에 시간을 추가하기 위한 datetime 라이브러리
import os  # 폴더 생성을 위해 추가
import re

import logging # log 출력을 위한 라이브러리 #TODO: logging.info 했을떄 출력 안되는 이유 찾기-> logging 사용법 조사

In [17]:
def object_detection_with_tracking():
    # Load the YOLOv8 model - object detection
    model = YOLO('yolov8n.pt')
    track_history = defaultdict(lambda: [])
    names = model.model.names
    
    # 사람 탐지 여부
    human_detected = False

    # # webcam 사용 시 - 실시간 개체 추적
    cap = cv2.VideoCapture(1)     # 사용할 camera index 사용 - 0 : 랩탑 카메라, 1 : 핸드폰 연결 카메라(슬 경우)

    # using sample video file
    # cap = cv2.VideoCapture(video_path)
    # assert cap.isOpened(), "Error reading video file"

    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # 현재 시간을 포맷에 맞게 문자열로 변환하여 파일명에 추가
    current_time = datetime.now().strftime("%d_%H%M%S")
    # 파일명에 현재 시간 추가
    file_name = f"real_time_detec_{current_time}.mp4"

    folder_path = "./result"

    # 만약 폴더가 존재하지 않으면 폴더 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # VideoWriter 객체 생성
    result = cv2.VideoWriter(os.path.join(folder_path, file_name),  
                                cv2.VideoWriter_fourcc(*'mp4v'),      # macOS MP4 코덱인 MP4V로 FourCC 코드 설정
                                fps,
                                (w, h))

    recognition_start_time = {}

    # 카메라 / 비디오로부터 프레임 읽어오기
    while cap.isOpened():
        # 비디오의 각 프레임마다 객체 추적을 수행
        success, frame = cap.read()
        
        # for debugging
        cv2.imshow('realtime_video', frame)
        
        # success 변수가 True일 때 (비디오 프레임을 제대로 읽어왔을 때)
        while success:
            '''
            실시간 객체 탐지
            '''
            print("Start Object Detecting")
            results = model(frame, stream=True)
            
            # 사람 tracking
            human_tracking = model.track(frame, persist=True, verbose=False, classes=0)
            boxes = human_tracking[0].boxes.xyxy.cpu()
            
            # 사람이 감지되면
            while human_tracking[0].boxes.id is not None:
                print("Human detected!")
                human_detected = True
                # Extract prediction results
                clss = human_tracking[0].boxes.cls.cpu().tolist()   # 추적된 객체의 class label
                track_ids = human_tracking[0].boxes.id.int().cpu().tolist()   # 추적된 객체의 id
                confs = human_tracking[0].boxes.conf.float().cpu().tolist()   # 추적된 객체의 신뢰도 점수 (정확성)

                # Annotator Init
                annotator = Annotator(frame, line_width=2)

                for box, cls, track_id in zip(boxes, clss, track_ids):
                    # 현재 시각을 해당 객체 인식 시작 시각으로 업데이트
                    if track_id not in recognition_start_time:
                        recognition_start_time[track_id] = time()
                    annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])    # 프레임에 BBox 그리기. 

                    # 해당 객체의 추적 이력을 저장
                    track = track_history[track_id]    # track_history 딕셔너리에서 해당 객체의 추적 이력을 가져오기
                    track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))     # 추적 이력에 현재 객체의 중심점 좌표를 추가. 중심점 좌표는 상자의 네 꼭짓점 좌표를 사용하여 계산.
                    if len(track) > 30:
                        track.pop(0)     # 추적 이력의 길이가 30보다 크다면, 가장 오래된 추적 이력을 제거. (= 각 객체에 대해 최근 30프레임 동안의 이동 경로를 추적하고 기록한다)

                    # 추적 이력을 사용하여 추적 경로 그리기 
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))     # 추적 이력을 넘파이 배열로 변환
                    cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)    # cv2.circle() 함수를 사용하여 추적 경로의 마지막 점에 원을 그리기
                    cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)   # cv2.polylines() 함수를 사용하여 추적 경로를 선으로 그리기

                # 객체 인식 기간(시간) 계산 및 출력    
                for track_id in recognition_start_time:
                    if track_id in track_ids:
                        recognition_duration = time() - recognition_start_time[track_id]
                        
                    # 20초 이상 탐지되는 객체의 id 출력
                    if recognition_duration >= 20 :
                        print(f"Object with track ID {track_id} recognized for {recognition_duration} seconds.")
                        #TODO: push 메세지 요청
                        
            result.write(frame)

            # 사람 사라지면 녹화 중단
            if human_tracking[0].boxes.id is None:
                print("Human disappered!")
                break
        # else:
        #     break

    result.release()
    cap.release()
    cv2.destroyAllWindows()
    
    return result



In [18]:
try1 = object_detection_with_tracking()

Start Object Detecting
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
Human detected!
H

KeyboardInterrupt: 

: 

- 사람이 감지되면 녹화를 시작하고 사람이 사라지면 녹화한 비디오를 저장하고 
- 다시 새로 사람이 감지되면 다시 녹화를 시작해야하는데 Human detected!부분에서 무한 반복되는듯..?
    - while문이 잘못 만들어짐? if문으로 수정

## 시도2.
- while 말고 if로 수정해서 일단 한번 사이클이 잘 도는지 확인

In [2]:
def object_detection_with_tracking():
    # Load the YOLOv8 model - object detection
    model = YOLO('yolov8n.pt')
    track_history = defaultdict(lambda: [])
    names = model.model.names
    
    # 사람 탐지 여부
    human_detected = False

    # # webcam 사용 시 - 실시간 개체 추적
    cap = cv2.VideoCapture(1)     # 사용할 camera index 사용 - 0 : 랩탑 카메라, 1 : 핸드폰 연결 카메라(슬 경우)

    # using sample video file
    # cap = cv2.VideoCapture(video_path)
    # assert cap.isOpened(), "Error reading video file"

    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # 현재 시간을 포맷에 맞게 문자열로 변환하여 파일명에 추가
    current_time = datetime.now().strftime("%d_%H%M%S")
    # 파일명에 현재 시간 추가
    file_name = f"real_time_detec_{current_time}.mp4"

    folder_path = "./result"

    # 만약 폴더가 존재하지 않으면 폴더 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # VideoWriter 객체 생성
    result = cv2.VideoWriter(os.path.join(folder_path, file_name),  
                                cv2.VideoWriter_fourcc(*'mp4v'),      # macOS MP4 코덱인 MP4V로 FourCC 코드 설정
                                fps,
                                (w, h))

    recognition_start_time = {}

    # 카메라 / 비디오로부터 프레임 읽어오기
    while cap.isOpened():
        # 비디오의 각 프레임마다 객체 추적을 수행
        success, frame = cap.read()
        
        # for debugging
        cv2.imshow('realtime_video', frame)
        
        # success 변수가 True일 때 (비디오 프레임을 제대로 읽어왔을 때)
        if success:
            '''
            실시간 객체 탐지
            '''
            print("Start Object Detecting")
            results = model(frame, stream=True)
            
            # 사람 tracking
            human_tracking = model.track(frame, persist=True, verbose=False, classes=0)
            boxes = human_tracking[0].boxes.xyxy.cpu()
            
            # 사람이 감지되면
            if human_tracking[0].boxes.id is not None:
                print("Human detected!")
                human_detected = True
                # Extract prediction results
                clss = human_tracking[0].boxes.cls.cpu().tolist()   # 추적된 객체의 class label
                track_ids = human_tracking[0].boxes.id.int().cpu().tolist()   # 추적된 객체의 id
                confs = human_tracking[0].boxes.conf.float().cpu().tolist()   # 추적된 객체의 신뢰도 점수 (정확성)

                # Annotator Init
                annotator = Annotator(frame, line_width=2)

                for box, cls, track_id in zip(boxes, clss, track_ids):
                    # 현재 시각을 해당 객체 인식 시작 시각으로 업데이트
                    if track_id not in recognition_start_time:
                        recognition_start_time[track_id] = time()
                    annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])    # 프레임에 BBox 그리기. 

                    # 해당 객체의 추적 이력을 저장
                    track = track_history[track_id]    # track_history 딕셔너리에서 해당 객체의 추적 이력을 가져오기
                    track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))     # 추적 이력에 현재 객체의 중심점 좌표를 추가. 중심점 좌표는 상자의 네 꼭짓점 좌표를 사용하여 계산.
                    if len(track) > 30:
                        track.pop(0)     # 추적 이력의 길이가 30보다 크다면, 가장 오래된 추적 이력을 제거. (= 각 객체에 대해 최근 30프레임 동안의 이동 경로를 추적하고 기록한다)

                    # 추적 이력을 사용하여 추적 경로 그리기 
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))     # 추적 이력을 넘파이 배열로 변환
                    cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)    # cv2.circle() 함수를 사용하여 추적 경로의 마지막 점에 원을 그리기
                    cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)   # cv2.polylines() 함수를 사용하여 추적 경로를 선으로 그리기

                # 객체 인식 기간(시간) 계산 및 출력    
                for track_id in recognition_start_time:
                    if track_id in track_ids:
                        recognition_duration = time() - recognition_start_time[track_id]
                        
                    # 20초 이상 탐지되는 객체의 id 출력
                    if recognition_duration >= 20 :
                        print(f"Object with track ID {track_id} recognized for {recognition_duration} seconds.")
                        #TODO: push 메세지 요청
            
            # 사람 사라지면 녹화 중단
            else: 
                print("Human disappered!")
                result.write(frame)
                
        else:
            break

    result.release()
    cap.release()
    cv2.destroyAllWindows()
    
    return result

try2 = object_detection_with_tracking()

2024-03-27 12:24:27.485 python[85590:7292561] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Det

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


카메라 연결을 끊어서 break를 하려고 했는데 에러 발생
- `error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'`
    - 해당 에러는 imshow에 넘겨주는 이미지 파일에 문제가 있을 때 발생
    - 발생하는 원인은 1) 이미지 파일에 문제가 있거나 2) 이미지 파일이 존재하지 않거나 3) 이미지 파일의 경로가 잘못 설정되어 있는 경우 발생
        - 참고 [블로그](https://koosco.tistory.com/53)

## 시도 3. 카메라 연결을 끊었을 경우에 대한 예외처리를 추가

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
from time import time
from collections import defaultdict
from ultralytics.utils.plotting import Annotator, colors
import math 

from datetime import datetime  # 파일명에 시간을 추가하기 위한 datetime 라이브러리
import os  # 폴더 생성을 위해 추가
import re

def object_detection_with_tracking():
    # Load the YOLOv8 model - object detection
    model = YOLO('yolov8n.pt')
    track_history = defaultdict(lambda: [])
    names = model.model.names
    
    # 사람 탐지 여부
    human_detected = False

    # # webcam 사용 시 - 실시간 개체 추적
    cap = cv2.VideoCapture(1)     # 사용할 camera index 사용 - 0 : 랩탑 카메라, 1 : 핸드폰 연결 카메라(슬 경우)

    # using sample video file
    # cap = cv2.VideoCapture(video_path)
    # assert cap.isOpened(), "Error reading video file"

    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # 현재 시간을 포맷에 맞게 문자열로 변환하여 파일명에 추가
    current_time = datetime.now().strftime("%d_%H%M%S")
    # 파일명에 현재 시간 추가
    file_name = f"real_time_detec_{current_time}.mp4"

    folder_path = "./result"

    # 만약 폴더가 존재하지 않으면 폴더 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # VideoWriter 객체 생성
    result = cv2.VideoWriter(os.path.join(folder_path, file_name),  
                                cv2.VideoWriter_fourcc(*'mp4v'),      # macOS MP4 코덱인 MP4V로 FourCC 코드 설정
                                fps,
                                (w, h))

    recognition_start_time = {}

    # 카메라 / 비디오로부터 프레임 읽어오기
    while cap.isOpened():
        # 비디오의 각 프레임마다 객체 추적을 수행
        success, frame = cap.read()
        
        # success 변수가 True일 때 (비디오 프레임을 제대로 읽어왔을 때)
        if success:
            '''
            실시간 객체 탐지
            '''
            # for debugging
            cv2.imshow('realtime_video', frame)
            print("Start Object Detecting")
            results = model(frame, stream=True)
            
            # 사람 tracking
            human_tracking = model.track(frame, persist=True, verbose=False, classes=0)
            boxes = human_tracking[0].boxes.xyxy.cpu()
            
            # 사람이 감지되면
            if human_tracking[0].boxes.id is not None:
                print("Human detected!")
                human_detected = True
                # Extract prediction results
                clss = human_tracking[0].boxes.cls.cpu().tolist()   # 추적된 객체의 class label
                track_ids = human_tracking[0].boxes.id.int().cpu().tolist()   # 추적된 객체의 id
                confs = human_tracking[0].boxes.conf.float().cpu().tolist()   # 추적된 객체의 신뢰도 점수 (정확성)

                # Annotator Init
                annotator = Annotator(frame, line_width=2)

                for box, cls, track_id in zip(boxes, clss, track_ids):
                    # 현재 시각을 해당 객체 인식 시작 시각으로 업데이트
                    if track_id not in recognition_start_time:
                        recognition_start_time[track_id] = time()
                    annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])    # 프레임에 BBox 그리기. 

                    # 해당 객체의 추적 이력을 저장
                    track = track_history[track_id]    # track_history 딕셔너리에서 해당 객체의 추적 이력을 가져오기
                    track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))     # 추적 이력에 현재 객체의 중심점 좌표를 추가. 중심점 좌표는 상자의 네 꼭짓점 좌표를 사용하여 계산.
                    if len(track) > 30:
                        track.pop(0)     # 추적 이력의 길이가 30보다 크다면, 가장 오래된 추적 이력을 제거. (= 각 객체에 대해 최근 30프레임 동안의 이동 경로를 추적하고 기록한다)

                    # 추적 이력을 사용하여 추적 경로 그리기 
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))     # 추적 이력을 넘파이 배열로 변환
                    cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)    # cv2.circle() 함수를 사용하여 추적 경로의 마지막 점에 원을 그리기
                    cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)   # cv2.polylines() 함수를 사용하여 추적 경로를 선으로 그리기

                # 객체 인식 기간(시간) 계산 및 출력    
                for track_id in recognition_start_time:
                    if track_id in track_ids:
                        recognition_duration = time() - recognition_start_time[track_id]
                        
                    # 20초 이상 탐지되는 객체의 id 출력
                    if recognition_duration >= 20 :
                        print(f"Object with track ID {track_id} recognized for {recognition_duration} seconds.")
                        #TODO: push 메세지 요청
            
            # 사람 사라지면 녹화 중단
            else: 
                print("Human disappered!")
                result.write(frame)
                
        else:
            break

    result.release()
    cap.release()
    cv2.destroyAllWindows()
    
    return result

try3 = object_detection_with_tracking()

2024-03-27 12:30:20.862 python[86246:7303898] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Detecting
Human disappered!
Start Object Det

### 작동내용
- 카메라가 연결되면 object detection 유지, 카메라가 연결 해제되면 object detection 중단
- 사람 탐지 확인

**[문제점]**
- 카메라 연결 시점에서 녹화가 되고, 카메라 연결 해제될때 녹화가 중단됨
    - `cv2.VideoWriter`의 `write`, `release` 기능 정리!
- 녹화 속도가 빠름 
    - 원인?
- 실행됬던 커널이 종료가 안되고 멈춤 -> 계속 커널 종료후 재시작 반복 
    - 원인?
        - release 시점에 `sys.exit()` 추가 

---
#### `cv2.VideoWriter`
- cv2.VideoWriter : 비디오 출력 객체 생성
- [write()](https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html#a3115b679d612a6a0b5864a0c88ed4b39): Writes the next video frame.
    - 영상 녹화
- [release()](https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html#a667f737e56d5ba6b0533c6c7bf941140) : Closes the video writer
    - 비디오 출력 객체 해제? != 저장 종료
    - 저장하는 작업을 끝냈다는 의미
        - 한번 release하면 이 VideoWriter는 재사용 불가!
            - 객체를 생성하고 해제하는 사이클을 만들어야함

[참고자료]
- [opencv docs](https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html#a3115b679d612a6a0b5864a0c88ed4b39), [블로그](https://deep-learning-study.tistory.com/108), [블로그2](https://overface.tistory.com/584), claude.ai

---

## 시도 4. 에러 부분 수정
- release 시점에 `sys.exit()` 추가
- videoWriter: 사람이 감지된 이후에 생성되도록 수정

In [5]:
import cv2
from ultralytics import YOLO
import numpy as np
from time import time
from collections import defaultdict
from ultralytics.utils.plotting import Annotator, colors
import math

from datetime import datetime  # 파일명에 시간을 추가하기 위한 datetime 라이브러리
import os  # 폴더 생성을 위해 추가
import re

import sys

def object_detection_with_tracking():
    # Load the YOLOv8 model - object detection
    model = YOLO('yolov8n.pt')
    track_history = defaultdict(lambda: [])
    names = model.model.names

    # 사람 탐지 여부
    human_detected = False

    # webcam 사용 시 - 실시간 개체 추적
    cap = cv2.VideoCapture(1)     # 사용할 camera index 사용 - 0 : 랩탑 카메라, 1 : 핸드폰 연결 카메라(슬 경우)

    # 웹캠 예외처리
    if not cap.isOpened():
        print("Camera open failed!")
        sys.exit()
    
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # VideoWriter 객체 생성 - 녹화를 시작하지 않음
    folder_path = "./result"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    recognition_start_time = {}

    # 카메라 / 비디오로부터 프레임 읽어오기
    while True:
        # 비디오의 각 프레임마다 객체 추적을 수행
        ret, frame = cap.read()
        if not ret :
            print("ret is not here!")
            break

        # ret 변수가 True일 때 (비디오 프레임을 제대로 읽어왔을 때)
        # if ret:
        '''
        실시간 객체 탐지
        '''
        # for debugging
        # cv2.imshow('realtime_video', frame)
        print("Start Object Detecting")
        results = model(frame, stream=True)

        # 사람 tracking
        human_tracking = model.track(frame, persist=True, verbose=False, classes=0)
        boxes = human_tracking[0].boxes.xyxy.cpu()

        # 사람이 감지되면
        if human_tracking[0].boxes.id is not None:
            print("Human detected!")

            # 사람이 처음 감지되었다면, 녹화 시작
            if not human_detected:
                human_detected = True
                current_time = datetime.now().strftime("%d_%H%M%S")
                file_name = f"real_time_detec_{current_time}.mp4"
                video_saving = cv2.VideoWriter(os.path.join(folder_path, file_name),
                                         cv2.VideoWriter_fourcc(*'mp4v'),
                                         fps,
                                         (w, h))
                
                # excetpion handling : VideoWriter 파일이 안 열린경우
                if not video_saving.isOpened():
                    print('File open failed!')
                    cap.release()
                    sys.exit()

            # Extract prediction results
            clss = human_tracking[0].boxes.cls.cpu().tolist()   # 추적된 객체의 class label
            track_ids = human_tracking[0].boxes.id.int().cpu().tolist()   # 추적된 객체의 id
            confs = human_tracking[0].boxes.conf.float().cpu().tolist()   # 추적된 객체의 신뢰도 점수 (정확성)

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                # 현재 시각을 해당 객체 인식 시작 시각으로 업데이트
                if track_id not in recognition_start_time:
                    recognition_start_time[track_id] = time()
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])    # 프레임에 BBox 그리기.

                # 해당 객체의 추적 이력을 저장
                track = track_history[track_id]    # track_history 딕셔너리에서 해당 객체의 추적 이력을 가져오기
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))     # 추적 이력에 현재 객체의 중심점 좌표를 추가. 중심점 좌표는 상자의 네 꼭짓점 좌표를 사용하여 계산.
                if len(track) > 30:
                    track.pop(0)     # 추적 이력의 길이가 30보다 크다면, 가장 오래된 추적 이력을 제거. (= 각 객체에 대해 최근 30프레임 동안의 이동 경로를 추적하고 기록한다)

                # # 추적 이력을 사용하여 추적 경로 그리기
                # points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))     # 추적 이력을 넘파이 배열로 변환
                # cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)    # cv2.circle() 함수를 사용하여 추적 경로의 마지막 점에 원을 그리기
                # cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)   # cv2.polylines() 함수를 사용하여 추적 경로를 선으로 그리기

            # 객체 인식 기간(시간) 계산 및 출력
            for track_id in recognition_start_time:
                if track_id in track_ids:
                    recognition_duration = time() - recognition_start_time[track_id]

                # 20초 이상 탐지되는 객체의 id 출력
                if recognition_duration >= 20:
                    print(f"Object with track ID {track_id} recognized for {recognition_duration} seconds.")
                    # TODO: push 메세지 요청

            # 사람이 감지되면 프레임을 비디오에 쓰기
            video_saving.write(frame)

        # 사람이 사라지면 녹화 중단 및 비디오 저장
        else:
            if human_detected:
                print("Human disappered!")
                human_detected = False
                video_saving.release()
                # sys.exit()

        # # 1초마다 사용자 입력을 기다림  => 주피터 노트북에서는 인식을 못함
        # if cv2.waitKey(int(1000/fps)) == 27: # 27은 esc키 번호
        #     break
        
    cap.release()
    cv2.destroyAllWindows()

try4 = object_detection_with_tracking()

Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detect

KeyboardInterrupt: 

## 시도 5. 사람 사라지고 일정시간 뒤에 녹화 종료
- 사람 detection이 끝나자마자 녹화를 종료하니 사람이 사라지는 순간(detection 에는 사라졌지만)에 비디오 녹화가 종료됨
- 사람이 사라지고나서 설정한 시간이 지나고나면 VideoWriter 객체 release

In [2]:
import cv2
from ultralytics import YOLO
import numpy as np
from time import time
from collections import defaultdict
from ultralytics.utils.plotting import Annotator, colors
import math

from datetime import datetime  # 파일명에 시간을 추가하기 위한 datetime 라이브러리
import os  # 폴더 생성을 위해 추가
import re

import sys

def object_detection_with_tracking():
    # Load the YOLOv8 model - object detection
    model = YOLO('yolov8n.pt')
    track_history = defaultdict(lambda: [])
    names = model.model.names

    # 사람 탐지 여부
    human_detected = False
    human_disappeared_time = None

    # webcam 사용 시 - 실시간 개체 추적
    cap = cv2.VideoCapture(1)     # 사용할 camera index 사용 - 0 : 랩탑 카메라, 1 : 핸드폰 연결 카메라(슬 경우)

    # 웹캠 예외처리
    if not cap.isOpened():
        print("Camera open failed!")
        sys.exit()
    
    # w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    w, h = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30 # 초당 프레임 30

    # VideoWriter 객체 생성 - 녹화를 시작하지 않음
    folder_path = "./result"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    recognition_start_time = {}

    # 카메라 / 비디오로부터 프레임 읽어오기
    while True:
        # 비디오의 각 프레임마다 객체 추적을 수행
        ret, frame = cap.read()
        if not ret :
            print("ret is not here!")
            break

        # ret 변수가 True일 때 (비디오 프레임을 제대로 읽어왔을 때)
        # if ret:
        '''
        실시간 객체 탐지
        '''
        # for debugging
        # cv2.imshow('realtime_video', frame)
        print("Start Object Detecting")
        results = model(frame, stream=True)

        # 사람 tracking
        human_tracking = model.track(frame, persist=True, verbose=False, classes=0)
        boxes = human_tracking[0].boxes.xyxy.cpu()

        # 사람이 감지되면
        if human_tracking[0].boxes.id is not None:
            print("Human detected!")

            # 사람이 처음 감지되었다면, 녹화 시작
            if not human_detected:
                human_detected = True
                current_time = datetime.now().strftime("%d_%H%M%S")
                file_name = f"real_time_detec_{current_time}.mp4"
                video_saving = cv2.VideoWriter(os.path.join(folder_path, file_name),
                                         cv2.VideoWriter_fourcc(*'mp4v'),
                                         fps,
                                         (w, h))
                human_disappeared_time = None
                # excetpion handling : VideoWriter 파일이 안 열린경우
                if not video_saving.isOpened():
                    print('File open failed!')
                    cap.release()
                    sys.exit()

            # Extract prediction results
            clss = human_tracking[0].boxes.cls.cpu().tolist()   # 추적된 객체의 class label
            track_ids = human_tracking[0].boxes.id.int().cpu().tolist()   # 추적된 객체의 id
            confs = human_tracking[0].boxes.conf.float().cpu().tolist()   # 추적된 객체의 신뢰도 점수 (정확성)

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                # 현재 시각을 해당 객체 인식 시작 시각으로 업데이트
                if track_id not in recognition_start_time:
                    recognition_start_time[track_id] = time()
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])    # 프레임에 BBox 그리기.

                # 해당 객체의 추적 이력을 저장
                track = track_history[track_id]    # track_history 딕셔너리에서 해당 객체의 추적 이력을 가져오기
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))     # 추적 이력에 현재 객체의 중심점 좌표를 추가. 중심점 좌표는 상자의 네 꼭짓점 좌표를 사용하여 계산.
                if len(track) > 30:
                    track.pop(0)     # 추적 이력의 길이가 30보다 크다면, 가장 오래된 추적 이력을 제거. (= 각 객체에 대해 최근 30프레임 동안의 이동 경로를 추적하고 기록한다)

                # # 추적 이력을 사용하여 추적 경로 그리기
                # points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))     # 추적 이력을 넘파이 배열로 변환
                # cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)    # cv2.circle() 함수를 사용하여 추적 경로의 마지막 점에 원을 그리기
                # cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)   # cv2.polylines() 함수를 사용하여 추적 경로를 선으로 그리기

            # 객체 인식 기간(시간) 계산 및 출력
            for track_id in recognition_start_time:
                if track_id in track_ids:
                    recognition_duration = time() - recognition_start_time[track_id]

                # 20초 이상 탐지되는 객체의 id 출력
                if recognition_duration >= 20:
                    print(f"Object with track ID {track_id} recognized for {recognition_duration} seconds.")
                    # TODO: push 메세지 요청

            # 사람이 감지되면 프레임을 비디오에 쓰기
            video_saving.write(frame)

        # 사람이 사라지면 사라진 시간(human_disappeared_time) 기록, human_detected 변수 False로 수정
        else:
            if human_detected:
                print("Human disappered!")
                human_disappeared_time = time()
                human_detected = False

        # 사람이 사라지고 10초가 지난뒤에 저장객체 해제, 사람 사라진 시간 변수(human_disappeared_time) 초기화
        if human_disappeared_time and (time() - human_disappeared_time) >= 10:
            video_saving.release()
            human_disappeared_time = None
                
        # # 1초마다 사용자 입력을 기다림  => 주피터 노트북에서는 인식을 못함
        # if cv2.waitKey(int(1000/fps)) == 27: # 27은 esc키 번호
        #     break

        
    cap.release()
    cv2.destroyAllWindows()

try5 = object_detection_with_tracking()

Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Det

KeyboardInterrupt: 

- 현재 저장된 영상을 재생하면 속도가 빠른재생처럼 빠름
- fps를 30으로 설정하면 1초에 30개 프레임이 재생되는 영상을 만드는것
- [블로그](https://deep-eye.tistory.com/10)를 참고해서 time 함수를 이용해 fps를 설정해보기

## 시도 6. fps 설정
- 재생속도가 일반 속도와 같도록 수정

In [9]:
import cv2
from ultralytics import YOLO
import numpy as np
from time import time
from collections import defaultdict
from ultralytics.utils.plotting import Annotator, colors
import math

from datetime import datetime  # 파일명에 시간을 추가하기 위한 datetime 라이브러리
import os  # 폴더 생성을 위해 추가
import re

import sys

def object_detection_with_tracking():
    # Load the YOLOv8 model - object detection
    model = YOLO('yolov8n.pt')
    track_history = defaultdict(lambda: [])
    names = model.model.names

    # 사람 탐지 여부
    human_detected = False
    human_disappeared_time = None

    # webcam 사용 시 - 실시간 개체 추적
    cap = cv2.VideoCapture(1)     # 사용할 camera index 사용 - 0 : 랩탑 카메라, 1 : 핸드폰 연결 카메라(슬 경우)

    # 웹캠 예외처리
    if not cap.isOpened():
        print("Camera open failed!")
        sys.exit()
    
    # w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    w, h = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30 # 초당 프레임 30 for most webcams
    prev_time = 0
    
    # VideoWriter 객체 생성 - 녹화를 시작하지 않음
    folder_path = "./result"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    recognition_start_time = {}

    # 카메라 / 비디오로부터 프레임 읽어오기
    while True:
        # 비디오의 각 프레임마다 객체 추적을 수행
        ret, frame = cap.read()
        
        # 경과 시간 = 현재 시간 - 이전 프레임 재생 시간 
        current_time = time() - prev_time
        
        if not ret :
            print("ret is not here!")
            break
        
        if (ret is True) and (current_time > 1./fps) :
            
            prev_time = time()
            
            '''
            실시간 객체 탐지
            '''
            # for debugging
            # cv2.imshow('realtime_video', frame)
            print("Start Object Detecting")
            results = model(frame, stream=True)
    
            # 사람 tracking
            human_tracking = model.track(frame, persist=True, verbose=False, classes=0)
            boxes = human_tracking[0].boxes.xyxy.cpu()
    
            # 사람이 감지되면
            if human_tracking[0].boxes.id is not None:
                print("Human detected!")
    
                # 사람이 처음 감지되었다면, 녹화 시작
                if not human_detected:
                    human_detected = True
                    current_time = datetime.now().strftime("%d_%H%M%S")
                    file_name = f"real_time_detec_{current_time}.mp4"
                    video_saving = cv2.VideoWriter(os.path.join(folder_path, file_name),
                                             cv2.VideoWriter_fourcc(*'mp4v'),
                                             fps,
                                             (w, h))
                    human_disappeared_time = None

                    
                    # excetpion handling : VideoWriter 파일이 안 열린경우
                    if not video_saving.isOpened():
                        print('File open failed!')
                        cap.release()
                        sys.exit()
    
                # Extract prediction results
                clss = human_tracking[0].boxes.cls.cpu().tolist()   # 추적된 객체의 class label
                track_ids = human_tracking[0].boxes.id.int().cpu().tolist()   # 추적된 객체의 id
                confs = human_tracking[0].boxes.conf.float().cpu().tolist()   # 추적된 객체의 신뢰도 점수 (정확성)
    
                # Annotator Init
                annotator = Annotator(frame, line_width=2)
    
                for box, cls, track_id in zip(boxes, clss, track_ids):
                    # 현재 시각을 해당 객체 인식 시작 시각으로 업데이트
                    if track_id not in recognition_start_time:
                        recognition_start_time[track_id] = time()
                    annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])    # 프레임에 BBox 그리기.
    
                    # 해당 객체의 추적 이력을 저장
                    track = track_history[track_id]    # track_history 딕셔너리에서 해당 객체의 추적 이력을 가져오기
                    track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))     # 추적 이력에 현재 객체의 중심점 좌표를 추가. 중심점 좌표는 상자의 네 꼭짓점 좌표를 사용하여 계산.
                    if len(track) > 30:
                        track.pop(0)     # 추적 이력의 길이가 30보다 크다면, 가장 오래된 추적 이력을 제거. (= 각 객체에 대해 최근 30프레임 동안의 이동 경로를 추적하고 기록한다)
    
                    # # 추적 이력을 사용하여 추적 경로 그리기
                    # points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))     # 추적 이력을 넘파이 배열로 변환
                    # cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)    # cv2.circle() 함수를 사용하여 추적 경로의 마지막 점에 원을 그리기
                    # cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)   # cv2.polylines() 함수를 사용하여 추적 경로를 선으로 그리기
    
                # 객체 인식 기간(시간) 계산 및 출력
                for track_id in recognition_start_time:
                    if track_id in track_ids:
                        recognition_duration = time() - recognition_start_time[track_id]
    
                    # 20초 이상 탐지되는 객체의 id 출력
                    if recognition_duration >= 20:
                        print(f"Object with track ID {track_id} recognized for {recognition_duration} seconds.")
                        # TODO: push 메세지 요청
    
                # 사람이 감지되면 프레임을 비디오에 쓰기
                video_saving.write(frame)
    
            # 사람이 사라지면 사라진 시간(human_disappeared_time) 기록, human_detected 변수 False로 수정
            else:
                if human_detected:
                    print("Human disappered!")
                    human_disappeared_time = time()
                    human_detected = False
    
            # 사람이 사라지고 10초가 지난뒤에 저장객체 해제, 사람 사라진 시간 변수(human_disappeared_time) 초기화
            if human_disappeared_time and time() - human_disappeared_time >= 10:
                video_saving.release()
                human_disappeared_time = None
                    
            # # 1초마다 사용자 입력을 기다림  => 주피터 노트북에서는 인식을 못함
            # if cv2.waitKey(int(1000/fps)) == 27: # 27은 esc키 번호
            #     break

        
    cap.release()
    cv2.destroyAllWindows()

try6 = object_detection_with_tracking()

Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start Object Detecting
Human detected!
Start 

KeyboardInterrupt: 

---
### SUMMARY : 지금까지의 작업 완료 내용
- 카메라가 켜지면 : object detecting 시작+유지
    - 카메라가 꺼지면 자동으로 실행종료
        - 문제 : 프로그램이 실행중인 커널은 자동 종료가 안되고 카메라만 종료되는듯 
            - 커널을 재시작해야 프로그램 재시작 가능
            - 커널 종료 안하고 바로 프로그램 실행 -> 카메라 ret을 못 잡아냄
- 사람이 detecting되면(annotate person이 보이면) : 녹화시작(파일생성)
    - 사람이 사라지고 10초 뒤 저장(생성된 파일에 비디오 저장됨)
